# State Machines in Python — Part 1

### Installing STMPY

If you are running this notebook on Binder, STMPY should already be installed and you can jump to the next cell.

If you haven't installed stmpy, install it via the following command line commands:

`python3 -m pip install --upgrade stmpy`

Once you have done this  the command line, come back to this notebook and restart the kernel. (Grey menu bar at the top of this page, Kernel / Restart.) Then run the cell above again. Alternatively, you can also run the following cell which executes the command for you.

In [ ]:
!python3 -m pip install --upgrade stmpy

### Checking the Installation

Run the following cell by clicking `Shift` + `Enter`. It should output the current version of stmpy you have installed. If it prints `ModuleNotFoundError: No module named 'stmpy'` it means the Python interpreter does not have the STMPY library installed.

In [ ]:
import stmpy

print("STMPY Version installed: {}".format(stmpy.__version__))

Sometimes, you have several Python interpreters on your machine. You can check on which one notebooks run with the code below. It will show you which Python interpreter is used. You should use the corresponding pip command to install the notebooks.

In [ ]:
import sys
sys.executable

> If you have troubles getting notebooks with STMPY to run, take contact with us!

# Getting Started - Step by Step

*In the following we go through the setup of a single state machine, almost line by line so all details are covered. Make sure to execute every notebook cell with Python code in it, and make sure it executes correctly without error message. The next tutorials will present code in a more compact way. So if you feel confident in Python and think this goes to slow, there's hope. If you struggle a bit with Python, have an extra close look at the details.*

Let's start with a simple state machine:

<img src="images/ticktock.png" style="max-width:100%;" />

The state machine calls method `on_init()` when it starts, and goes into state `s_tick`. Then, it toggles back and forth to the `s_tock` state, controlled by timers. 


### Step 1: Python Class for Actions

The actions in the transitions directly refer to Python methods `on_init()`, `on_tick()`, `on_tock()`. We declare them in a class `Tick`. This class works just as a warpper around these methods, and is a container for the variabes `ticks` and `tocks`.

In [ ]:
from IPython.display import clear_output


class Tick:

    def on_init(self):
        clear_output(wait=True)
        print('Init!')
        # self.stm.diver.print_status()
        self.ticks = 0
        self.tocks = 0

    def on_tick(self):
        clear_output(wait=True)
        print('Tiick! {}'.format(self.ticks))
        print(self.stm.driver.print_status())
        self.ticks = self.ticks + 1

    def on_tock(self):
        clear_output(wait=True)
        print('Tooock! {}'.format(self.tocks))
        print(self.stm.driver.print_status())
        self.tocks = self.tocks + 1

Above you can see the three methods the state machine refers to. Within their body, they control simple counter variables `ticks` and `tocks`, but in principle you can do anything in these methods you like. 

We also need an instance of the Tick class for later:

In [ ]:
tick = Tick()

### Step 2: Declaring Transitions and States

**Transitions:** We declare the logic of the state machines by creating Python dictionaries for each of the transitions above. They declare source state, target state, trigger (unless its an initial transition), and effects. The effects declare a set of actions, separated by a `;`. Some of the actions refer to the methods defined in the class `Tick` from above, the others start timers directly. 

Note that the action `"on_init; start_timer('tick', 300)"` uses `'` to surround the string argument `tick` since it is declared within a Python string itself encalpsulated by `"`. (The opposite, that means `'on_init; start_timer("tick", 300)'` would also work.)

In [ ]:
# initial transition
t0 = {
    "source": "initial",
    "target": "s_tick",
    "effect": "on_init; start_timer('tick', 300)",
}

# transition s_tick ----> s_tock
t1 = {
    "trigger": "tick",
    "source": "s_tick",
    "target": "s_tock",
    "effect": "on_tick; start_timer('tock', 300)",
}

# transition s_tock ----> s_tick
t2 = {
    "trigger": "tock",
    "source": "s_tock",
    "target": "s_tick",
    "effect": "on_tock; start_timer('tick', 300)",
}

In this example, we don't have to declare anything special for the states, so they are only declared indirectly as values in the dictionary of the transitions. (In a later example we see how they can have entry and exit actions.)

### Step 3: Creating the State Machine

First, we need to import the Machine class. It contains code to execute the transitions and take care of all other stuff related to the state machine.

In [ ]:
from stmpy import Machine

We declare an instance for the machine, passing it the `tick` object from above and the transitions. It also gets a name.

In [ ]:
tick_tock_machine = Machine(transitions=[t0, t1, t2], obj=tick, name="tick_tock")

Now we need to do one more technical thing. Sometimes the Python class that implements our actions wants to send messages or manipulate timers, or get access to the STMPY API to do other things. For this, we use variable `stm` in the Tick class. We have to set the value of this variable now that we have created the corresponding machine. (For this example it is not necessary, but we want to introduce this step from the beginning.)

In [ ]:
tick.stm = tick_tock_machine

### Step 4: Adding the State Machine to a Driver, and Start!

The state machine is declared and ready, we only have to run it. State machines are not executed directly, but assigned to a **Driver**. One driver corresponds to one thread (or process) and can execute many state machines at the same time. 

In [ ]:
from stmpy import Driver

driver = Driver()

# add our state machine to the driver
driver.add_machine(tick_tock_machine)

Now the driver is declared. What's left is to start it. Since this machine is describing an endless loop, we limit the number of transitions the driver executes to 5. You will see that the notebook cell is active until the driver stops.

In [ ]:
# start the driver, with limited number of transitions
driver.start(max_transitions=10)

Execute all code cells above, step by step. The last one should start the state machine. You should now see that the state machine prints the following under the cell above:

    Init!
    Tick! 0
    Tock! 0
    Tick! 1
    Tock! 1
    
You can also observe that the transitions are triggered by the timers, one every 1000 milliseconds.

# Repetition

- Transitions are declared as Python dictionaries.
- Actions on transitions are declared in a special class for that state machine.
- A `Machine` represents a state machine.
- A `Driver` is needed to execute one (or several) state machines.

In [ ]:
driver.stop()